In [177]:
import requests
from pymongo import MongoClient
import time
from datetime import datetime

# Function to convert ISO date to Unix timestamp
def iso_to_unix(iso_date):
    return int(datetime.fromisoformat(iso_date).timestamp())

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['opensea_db']
collection = db['cryptopunks_events']
collection.delete_many({})

# API endpoint
collection_slug = 'cryptopunks'  # The slug for CryptoPunks
api_url = f'https://api.opensea.io/api/v2/events/collection/{collection_slug}'

# Date range in ISO format
occurred_after = '2021-08-31T22:00:00'  # Start of time range
occurred_before = '2021-09-30T22:00:00'  # End of time range

# Convert to Unix timestamps
after_timestamp = iso_to_unix(occurred_after)
before_timestamp = iso_to_unix(occurred_before)

# Adjusted Parameters
params = {
    'after': after_timestamp,  # Unix timestamp for start of time range
    'before': before_timestamp,  # Unix timestamp for end of time range
    'event_type': 'all',  # Event type to retrieve
    'limit': '50',  # Max limit per request
}

# Headers (include your API key)
headers = {
    'Accept': 'application/json',
    'X-API-KEY': 'fd22235ed7874ca4971d8b11fbcd7ff7',  # Replace with your actual API key
}

print("Starting data collection...")

while True:
    response = requests.get(api_url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        events = data.get('asset_events', [])
        if not events:
            print("No more events found.")
            break
        
        # Transform and insert events into MongoDB
        for event in events:
            # Convert total_price from wei to ether if present
            total_price_wei = event.get('total_price')
            if total_price_wei:
                total_price_eth = float(total_price_wei) / 1e18
                event['total_price_eth'] = total_price_eth
            else:
                event['total_price_eth'] = None
            
            # Insert into MongoDB
            collection.insert_one(event)
        
        # Handle pagination
        next_page = data.get('next')
        if next_page:
            params['cursor'] = next_page
            time.sleep(1)  # Sleep to respect API rate limits
        else:
            break
    elif response.status_code == 429:
        print("Rate limit exceeded. Waiting for 10 seconds.")
        time.sleep(10)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

print("Data collection complete.")


Starting data collection...


ConnectionError: HTTPSConnectionPool(host='api.opensea.io', port=443): Max retries exceeded with url: /api/v2/events/collection/cryptopunks?after=1630440000&before=1633032000&event_type=all&limit=50 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x117eb4a70>: Failed to resolve 'api.opensea.io' ([Errno 8] nodename nor servname provided, or not known)"))

In [172]:
# Count the total number of event transactions in the MongoDB collection
event_count = collection.count_documents({})

print(f"Total number of event transactions in the MongoDB collection: {event_count}")


Total number of event transactions in the MongoDB collection: 5894


In [175]:
# Aggregation to find the unique token identifiers and their maximum selling prices
most_expensive_punks = collection.aggregate([
    {
        "$match": {"payment.quantity": {"$ne": None}}  # Ensure payment.quantity exists
    },
    {
        "$group": {
            "_id": "$nft.identifier",  # Group by token ID
            "max_price": {"$max": "$payment.quantity"}  # Get the maximum price for each token ID
        }
    },
    {
        "$sort": {"max_price": -1}  # Sort by max_price descending
    },
    {
        "$limit": 10  # Limit to the top 10
    }
])

# Print results
print("Ten Most Expensive Punks:")
for punk in most_expensive_punks:
    # Convert payment quantity from wei to ether
    payment_quantity_wei = punk['max_price']
    payment_quantity_eth = float(payment_quantity_wei) / 1e18  # Convert wei to ether
    print(f"Token ID: {punk['_id']}, Selling Price (ETH): {payment_quantity_eth:.4f}")


Ten Most Expensive Punks:
Token ID: 1533, Selling Price (ETH): 123.2500
Token ID: 8375, Selling Price (ETH): 120.0000
Token ID: 6534, Selling Price (ETH): 119.9500
Token ID: 2888, Selling Price (ETH): 115.0000
Token ID: 7200, Selling Price (ETH): 115.0000
Token ID: 8322, Selling Price (ETH): 114.9900
Token ID: 2996, Selling Price (ETH): 114.5000
Token ID: 5123, Selling Price (ETH): 113.5000
Token ID: 2167, Selling Price (ETH): 113.0000
Token ID: 1417, Selling Price (ETH): 112.8800


In [176]:
from datetime import datetime

# Aggregation to count sales per day
sales_per_day = collection.aggregate([
    {
        "$match": {"event_type": "sale"}  # Filter for sale events
    },
    {
        "$group": {
            "_id": {
                "$dateToString": {"format": "%Y-%m-%d", "date": {
                    "$toDate": {
                        "$multiply": ["$event_timestamp", 1000]  # Ensure we are converting seconds to milliseconds
                    }
                }}  # Convert UNIX timestamp to date
            },
            "count": {"$sum": 1}  # Count the number of sales for each day
        }
    },
    {
        "$sort": {"count": 1}  # Sort by count ascending to find the least sales
    },
    {
        "$limit": 1  # Limit to the top 1 record
    }
])

# Print the result
for result in sales_per_day:
    print(f"Date with the least CryptoPunks sold: {result['_id']}, Number of sales: {result['count']}")


Date with the least CryptoPunks sold: 2021-09-30, Number of sales: 4631
